# Experiments with simple models and with using TensorBoard

Useful link: https://towardsdatascience.com/a-complete-guide-to-using-tensorboard-with-pytorch-53cb2301e8c3

In [22]:
from utils import *
import time
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import matplotlib.animation as animation
writergif = animation.PillowWriter(fps=6) 


In [23]:
def one_experiment(prob_delta = 0.1, num_segments = 2, pow_of_two = 4, max_iter = 2000, num_samples = 100000, dims_dense =[1, 2, 1]):
    run_name = f'NICE_simple_nn_scaling_eq_delta_{int(prob_delta*100)}'\
        f'_segments_{num_segments}_pow_two_{pow_of_two}'\
            f'_max_iter_{max_iter}_num_samples_{num_samples}_{int(time.time())}'\
                f'_dim_dense_{"-".join([str(i) for i in dims_dense])}' 
    path_to_tensorboard = f'runs2/' + run_name
    writer = SummaryWriter(path_to_tensorboard)

    # Creating models
    base = nf.distributions.base.DiagGaussian(2)
    flows = []
    for i in range(num_segments):
        flows += [AffineSingleBlock(SimpleDenseCustDim(dims=dims_dense))]
        flows += [Swap()]

    flows += [ScalingBlock(dim_x = 2)]
    model = MyNormFlow(q0 = base, flows = flows)

    #Writer for Tensorboard
    writer.add_graph(model, torch.tensor([[0.0, 0.0]]))
    writer.close()

    K_intervals = torch.Tensor([[0.5, 1],
                        [3,3]])

    nu = NormUnif(x_dim = 2, prob_delta=prob_delta, K_intervals=K_intervals)

    

    middle_point = [0,0]
    range_p = [5, 5]

    grid_size = 200
    xx, yy = torch.meshgrid(torch.linspace((middle_point[0]-range_p[0]), (middle_point[0]+range_p[0]), grid_size), torch.linspace((middle_point[1]-range_p[1]), (middle_point[1]+range_p[1]), grid_size))
    zz = torch.cat([xx.unsqueeze(2), yy.unsqueeze(2)], 2).view(-1, 2)


    # Train model
    show_iter = int(max_iter/8)


    loss_hist = np.array([])

    t_float = 0.3
    t = torch.tensor(t_float)
    true_prob = nu.prob_greater_t(t).item()
    optimizer = torch.optim.Adam(model.parameters(), lr=5e-4, weight_decay=1e-5)

    #animation
    # ims = []

    for it in tqdm(range(max_iter)):
        optimizer.zero_grad()
        
        # Get training samples
        x_np = nu.forward(num_samples)[0].numpy()
        x = torch.tensor(x_np).float()
        
        # Compute loss
        loss = model.forward_kld(x)
        
        # Do backprop and optimizer step
        if ~(torch.isnan(loss) | torch.isinf(loss)):
            loss.backward()
            optimizer.step()
        
        # Log loss
        loss_hist = np.append(loss_hist, loss.to('cpu').data.numpy())

        #error prob
        
        estim_prob = estim_prob_greater_t(model, 2*num_samples, t_float)
        
        
        writer.add_scalar('Loss', loss_hist[-1], it)
        writer.add_scalar('Bias_prob', estim_prob - true_prob, it)
        writer.add_scalar('SqErr_prob', (estim_prob - true_prob)**2, it)

        
        # Plot learned posterior
        if (it + 1) % show_iter == 0:
            model.eval()
            log_prob = model.log_prob(zz)
            model.train()
            prob = torch.exp(log_prob.to('cpu').view(*xx.shape))
            prob[torch.isnan(prob)] = 0

            fig = plt.figure(figsize=(15, 15))
            plt.pcolormesh(xx, yy, prob.data.numpy())
            plt.gca().set_aspect('equal', 'box')
            #plt.show()

            writer.add_figure('Distribution', fig, it)

            # #for animation
            # ims.append([fig])
            
    #animation
    # ani = animation.ArtistAnimation(fig, ims, interval=500, blit=True,
    #                             repeat_delay=1000)
    # animation_dir = "C:/Users/Sacia/OneDrive - Uniwersytet Wrocławski\Dokumenty/0. Studia/Matematyka/0. Praca magisterska/mgr_normalizing_flows/animation_folder/"+run_name+'.gif'
    # ani.save(animation_dir, writer=writergif)
    # Plot loss
    # plt.figure(figsize=(10, 10))
    # plt.plot(loss_hist, label='loss')
    # plt.legend()
    # plt.show()

    print({"max_iter":max_iter, 
                        "num_samples":num_samples,
                        'num_segments':num_segments,
                        'pow_of_two':pow_of_two,
                        'prob_delta': prob_delta,
                        'dims_dense':dims_dense},
                        {"loss": float(loss_hist[-1]), 
                         "Bias_prob_hyp":float(estim_prob - true_prob),
                         "SqErr_prob_hyp":float(estim_prob - true_prob)**2})


    writer.add_hparams({"max_iter":max_iter, 
                        "num_samples":num_samples,
                        'num_segments':num_segments,
                        'pow_of_two':pow_of_two,
                        'prob_delta': prob_delta,
                        'dims_dense':'-'.join([str(i) for i in dims_dense])},
                        {"loss": float(loss_hist[-1]), 
                         "Bias_prob_hyp":float(estim_prob - true_prob),
                         "SqErr_prob_hyp":float(estim_prob - true_prob)**2})

    writer.close()

In [24]:
a = np.array([4, 6, 8])  #segments
b = np.array([1]) #pow of two
c = np.array([2000, 4000]) #max iter
d = np.array([2048, 5000, 10000])
e = np.array([[1,16,64,16,1], [1,16,32,16,1], [1,4,32,4,1]])

ar = np.array([[xa, xb, xc, xd, xe] for xa in a for xb in b for xc in c for xd in d for xe in e])

<ipython-input-24-16f8de956280>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ar = np.array([[xa, xb, xc, xd, xe] for xa in a for xb in b for xc in c for xd in d for xe in e])


In [25]:
for num_segments, pow_of_two, max_iter, num_samples, dims_dense  in ar:
    print(num_segments, pow_of_two, max_iter, num_samples, dims_dense)
    one_experiment(prob_delta = 0.8, num_segments = int(num_segments), pow_of_two = int(pow_of_two), max_iter = int(max_iter), num_samples = int(num_samples), dims_dense = dims_dense.tolist())

4 1 2000 2048 [ 1 16 64 16  1]


 12%|█▏        | 247/2000 [00:11<01:43, 16.86it/s]<ipython-input-23-526ff04af551>:89: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  plt.pcolormesh(xx, yy, prob.data.numpy())
100%|██████████| 2000/2000 [01:26<00:00, 23.08it/s]


{'max_iter': 2000, 'num_samples': 2048, 'num_segments': 4, 'pow_of_two': 1, 'prob_delta': 0.8, 'dims_dense': [1, 16, 64, 16, 1]} {'loss': 3.037942886352539, 'Bias_prob_hyp': -0.004781603813171387, 'SqErr_prob_hyp': 2.2863735026135146e-05}
4 1 2000 2048 [ 1 16 32 16  1]


100%|██████████| 2000/2000 [01:07<00:00, 29.47it/s]


{'max_iter': 2000, 'num_samples': 2048, 'num_segments': 4, 'pow_of_two': 1, 'prob_delta': 0.8, 'dims_dense': [1, 16, 32, 16, 1]} {'loss': 3.05623197555542, 'Bias_prob_hyp': 0.007669568061828613, 'SqErr_prob_hyp': 5.882227425502151e-05}
4 1 2000 2048 [ 1  4 32  4  1]


...

{'max_iter': 4000, 'num_samples': 10000, 'num_segments': 8, 'pow_of_two': 1, 'prob_delta': 0.8, 'dims_dense': [1, 16, 64, 16, 1]} {'loss': 3.05100154876709, 'Bias_prob_hyp': -0.002493322563171363, 'SqErr_prob_hyp': 6.216657404019415e-06}
8 1 4000 10000 [ 1 16 32 16  1]


100%|██████████| 4000/4000 [09:47<00:00,  6.80it/s]


{'max_iter': 4000, 'num_samples': 10000, 'num_segments': 8, 'pow_of_two': 1, 'prob_delta': 0.8, 'dims_dense': [1, 16, 32, 16, 1]} {'loss': 3.067209482192993, 'Bias_prob_hyp': -0.007893322563171379, 'SqErr_prob_hyp': 6.230454108627039e-05}
8 1 4000 10000 [ 1  4 32  4  1]


100%|██████████| 4000/4000 [08:04<00:00,  8.26it/s]

{'max_iter': 4000, 'num_samples': 10000, 'num_segments': 8, 'pow_of_two': 1, 'prob_delta': 0.8, 'dims_dense': [1, 4, 32, 4, 1]} {'loss': 3.076395273208618, 'Bias_prob_hyp': 0.002856677436828603, 'SqErr_prob_hyp': 8.160605978085637e-06}
